## Clasificación de documentos:

# Calculo del género de una película a raíz de su sinopsis

## Parte I: Introducción

Se requiere la implementación de un algoritmo que, previamente entrenado, calcule el __género principal__ de una película en base al conocimiento adquirido.

Los géneros que tendrá en cuenta el algoritmo son: ___acción___, ___comedia___, ___terror___, ___bélico___ y __western__.

Los __archivos que contienen la sinopsis__ (o equivalente, pudiendo ser también un breve resumen de la primera parte de la película) de las películas estarán distribuídos de la siguiente forma: 1) Si forman parte del conjunto de pruebas estarán dentro de la carpeta del conjunto de prueba sin más, que será la carpeta en la que el algoritmo, una vez entrenado, buscará sinopsis para categorizarlas 2) Si forman parte del conjunto de entrenamiento estarán dentro de la carpeta del conjunto de entrenamiento y __a su vez__ dentro de una carpeta que indique su género.

Inicialmente se planteó utilizar _html_ como __formato__ para almacenar los archivos con los que vamos a trabajar pero, dado que no ha sido posible encontrar una fuente común para extraer todas las sinopsis, finalmente se almacenarán como __texto plano__, trabajando de esta manera con archivos _.txt_.

__Los conocimientos requeridos por parte del usuario__ que ejecutará el algoritmo __son mínimos__: tan sólo necesita __colocar__ los textos, en el formato adecuado, en la carpeta indicada y __ejecutar__ el algoritmo en sí. Únicamente se requiere mayor interacción por parte del usuario si desea cambiar el funcionamiento del algorimo en sí, o ajustar el mismo, como por ejemplo si desea cambiar las palabras clave para cada una de las categorías.

## Parte II: Escaneo de archivos

Empezamos símplemente indicando qué __carpeta__ contiene el __conjunto de entrenamiento__ y el de __prueba__, además de las __categorías__, que no serán útiles más adelante.

In [267]:
ruta_conjunto_entrenamiento = "conjunto_entrenamiento"
ruta_conjunto_prueba = "conjunto_prueba"

### Parte II-A: Escaneo de las categorías

Empezamos escaneando la carpeta del conjunto de entrenamiento, que contendrá las __categorías__ en las que se podrán clasificar los nuevos documentos:

In [268]:
import os # Nos ayudaremos de la librería "os" para leer ficheros y carpetas.

categorías = {elemento for elemento in os.listdir(ruta_conjunto_entrenamiento) if os.path.isdir(ruta_conjunto_entrenamiento + "/" + elemento)} # "os.listdir" devuelve las el contenido de un directorio dado, pero además queremos filtrar que sea un directorio, por eso lo procesamos y le aplicamos el filtro de que sea un directorio.

print(categorías)

{'j', 'ch'}


En la siguiente sección, vamos a recorrer nuestra estructura de carpetas para detectar cada uno de los textos a analizar.

### Parte II-B: Escaneo del conjunto de entrenamiento

Procedemos a __encontrar__ todos los __archivos ya clasificados__ (conjunto de entrenamiento), según su __categoría__.

Vamos a almacenar todos los archivos según su categoría en un diccionario, que contrendrá como clave la categoría y como valor un conjunto de archivos.

Además, los valores (archivos) del diccionario estarán contenidos en conjuntos (sets) ya que no nos interesa el orden y, además, no permite duplicados (no puede haber dos archivos con el mismo nombre).

In [269]:
archivos_entrenamiento = set()
archivos_entrenamiento_categoría = {}

for categoría in categorías:
    conjunto_auxiliar = set()
    
    for fichero in os.listdir(ruta_conjunto_entrenamiento + "/" + categoría):
        if fichero.endswith(".txt"):
            nombre_y_ruta_fichero = categoría + "/" + fichero
            conjunto_auxiliar.add(nombre_y_ruta_fichero)
    
    archivos_entrenamiento.update(conjunto_auxiliar)
    archivos_entrenamiento_categoría[categoría] = conjunto_auxiliar

Para __acceder a todos los archivos__, ejecutamos:

In [270]:
archivos_entrenamiento

{'ch/2 Fast 2 Furious - A todo gas 2.txt',
 'ch/A todo gas 3 punto 5 - Los bandoleros.txt',
 'ch/A todo gas 4.txt',
 'j/Apocalipsis now.txt'}

Y para obtener el __número total de archivos__ dentro __del conjunto de entrenamiento__ simplemente ejecutamos la siguiente instrucción:

In [271]:
print(len(archivos_entrenamiento))

4


Para __acceder a los archivos de una categoría__, por ejemplo acción, ejecutamos la siguiente orden:

In [272]:
archivos_entrenamiento_categoría["acción"]

Para obtener el __número de archivos dentro de una categoría__, ejecutamos:

In [273]:
len(archivos_entrenamiento_categoría["acción"])

### Parte II-C: Escaneo del conjunto de prueba

Y, a continuación, procedemos a __encontrar__ los __archivos que querríamos clasificar__ (conjunto de prueba).

Esta vez no es necesario usar diccionario (no necesitamos separarlos por categorías porque precisamente es lo que queremos hallar), así que usaremos un set por la misma razón por la que lo usamos como valor en el diccionario de los archivos del conjunto de entrenamiento (no queremos repeticiones ni nos interesa el orden).

In [274]:
archivos_prueba = set()
for file in os.listdir(ruta_conjunto_prueba):
    if file.endswith(".txt"):
        archivos_prueba.add(file)

print(archivos_prueba)

{'A todo gas - Tokyo race.txt'}


## Parte III: Elección del conjunto de palabras clave

Para ayudarnos en el estudio de las __palabras clave__ que debemos escoger para cada categoría vamos a realizar un pequeño estudio para determinar las palabras más frecuentes de cada categoría. La elección en sí debemos realizarla a mano puesto que no podemos escoger directamente las más frecuentes puesto que con total seguridad entre las más frecuentes se encontrarán verbos, conectores, preposiciones, pronombres, artículos, nombres propios, etc. que no nos serán de utilidad a la hora de determinar la categoría de una película.

Debemos definir __método que recibirá__ tanto __un conjunto de archivos__ como una __ruta__ donde se encuentran y __contará las palabras__ que aparecen en él __y el número de veces que dichas palabras aparecen__.

Pero antes, definiremos un par de métodos que nos será útiles.

El primero, convertido en método para aportar claridad al código, recibe una palabra y la procesa levemente (la convierte en minúsculas y elimina los carácteres no alfabéticos más típicos) para perder la menor cantidad de información posible (ya que más adelante el algoritmo descartará cualquier palabra que contenga carácteres que no sean alfabéticos).

In [275]:
def procesa_palabra(palabra):
    palabra = palabra.lower() # Pasamos la palabra a minúscula.
    # Para perder la menor información posible, reemplazamos ':', ',', ':' y ';', que son los carácteres más típicos que nos podemos encontrar adyacentes a una palabra y que la invalidarían en el siguiente if del algoritmo.
    palabra = palabra.replace('.', '')
    palabra = palabra.replace(',', '')
    palabra = palabra.replace(':', '')
    palabra = palabra.replace(';', '')
    
    return palabra

También definimos un método que reciba un solo archivo y cuente sus palabras. Este metodo será usado por el método que estamos buscando y los dividimos de esta forma porque debemos buscar en un solo archivo cuando apliquemos los algoritmos de Naive Bayes y kNN.

In [276]:
def cuenta_palabras_desde_archivo(ruta, archivo):
    cuenta_palabras = {}
    
    fichero = open(ruta + "/" + archivo, "r", encoding="latin-1") # Elegimos latin-1 en vez de utf-8 por problemas con las tildes.

    for palabra in fichero.read().split(): # Recorremos el fichero, palabra a palabra.
        palabra = procesa_palabra(palabra)
        if palabra.isalpha() is True: # Será true cuando todos los caracteres son alfabéticos y hay al menos uno.
            if palabra in cuenta_palabras:
                cuenta_palabras[palabra] += 1 # Si la palabra ya existe, entonces incrementa en 1 el número de veces que hace aparición.
            else:
                cuenta_palabras[palabra] = 1 # Si la palabra no existe, la añade (con valor 1 al número de veces que aparece).

    cuenta_palabras
    return cuenta_palabras

Ahora sí, definimos el método que estamos buscando en este momento y que se anunciaba antes:

In [277]:
from collections import Counter # Lo usaremos para añadir un diccionario a otro.

def cuenta_palabras_desde_archivos(ruta, archivos):
    cuenta_palabras = {}
    
    for archivo in archivos:
        new = cuenta_palabras_desde_archivo(ruta, archivo)
        cuenta_palabras = dict(Counter(cuenta_palabras)+Counter(new))
    
    return cuenta_palabras

Además, con propósito de limpiar la salida que obtendremos vamos a establecer un __umbral__ para desechar todas las palabras que se repitan por debajo del mismo:

In [278]:
umbral_repetición = 4

Y, con el mismo propósito, otro __umbral__ para desechar todas las palabras con una longitud menor a él:

In [279]:
umbral_longitud = 4

Y, por último, un método que use al anterior que, además, nos __ordene las palabras__ (de mayor a menor uso):

In [280]:
def cuenta_palabras_desde_archivos_ordenadas(ruta, archivos):
    cuenta_palabras = cuenta_palabras_desde_archivos(ruta, archivos)
    
    lista_ordenada = [] # Usamos una lista para poder ordenar las palabras.
    for palabra, contador in cuenta_palabras.items():
        lista_ordenada.append((contador, palabra)) # La lista ordenada almacenará una tupla.
    
    lista_ordenada = sorted(lista_ordenada, reverse = True) # Lo ordenamos y lo invertimos para que las palabras más frecuentes estén arriba.
    resultado = lista_ordenada.copy()
    
    for elemento in lista_ordenada:
        if (elemento[0] < umbral_repetición) or (len(elemento[1]) < umbral_longitud): # Si la palabra supera los umbrales indicados, se muestra.
            resultado.remove(elemento)
    
    return resultado

Palabras más frecuentes por categoría:

In [281]:
for categoría in categorías:
    print("----- Categoría [%s] -----" % (categoría))
    print(cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_categoría[categoría]))

----- Categoría [j] -----
[(1, 'tokyo'), (1, 'japon'), (1, 'chino')]
----- Categoría [ch] -----
[(5, 'chino'), (1, 'shanghai'), (1, 'pekin'), (1, 'macao')]


Una vez realizado el estudio, inicializamos manualmente las __palabras clave__ de cada __categoría__, usaremos unas 20 palabras para cada una de ellas (podrán repetirse entre categorías).

In [282]:
palabras_clave_categoría = {}

# Las inicializamos como conjuntos ya que no nos interesa el orden y, además, no se permiten duplicados.
palabras_clave_categoría["acción"] = {"policía", "coche", "agente", "auto", "escapar", "carrera", "seguridad", "llamada", "gasolina", "problemas", "operación", "escapa", "venganza", "prisión", "muerte", "hospital", "carreras", "ataque", "arma", "asesino", "asesina", "asesinos", "terrorista", "terroristas", "persecución", "muerto", "investigación", "destrucción", "bomba", "ruso"}
palabras_clave_categoría["comedia"] = {"incorrecto", "simpáticos", "alegre", "carcajadas", "cómica", "cómicos", "delirante", "divertidos", "irreverente", "risas", "sátira", "divertido", "divertida", "gags", "hilarantes", "humor", "tópicos", "crítica", "absurdo", "hilarante", "absurdas", "extravagante"}
palabras_clave_categoría["terror"] = {}
palabras_clave_categoría["bélico"] = {"guerra", "ejército", "sargento", "soldados", "general", "grupo", "hombres", "combate", "misión", "unidos", "matar", "teniente", "capitán", "alemán", "alemanes", "vietnam", "estadounidense", "estadounidenses", "soldado", "regimiento", "operación", "oficial", "infantería", "cabo", "enemigo"}
palabras_clave_categoría["western"] = {"pueblo", "hijo", "joven", "amigo", "sheriff", "pistolero", "banda", "guerra", "diligencia", "camino", "duelo", "bandidos", "granjero", "asesino", "revólver", "arma", "armas", "desierto", "oeste"}

Si es una categoría nueva y __no__ se han añadido palabras clave, vamos a generarlas automáticamente en base a su frecuencia. Para ello, necesitamos el siguiente método.

In [283]:
def genera_palabras_clave(categoría):
    lista_ordenada = cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_categoría[categoría])
    
    candidatos = lista_ordenada[:20] # Elegimos los 20 primeros elementos de la lista (no olvidemos que obtenemos una tupla).
    
    resultado = [elemento[1] for elemento in candidatos] # De la tupla, nos quedamos con el elemento "1", que contiene la cadena de texto que repesenta la palabra.
    
    return resultado

Antes de ejecutar el algoritmo para auto-generar las palabras claves, puesto que el resultado carecerá de revisión, vamos a __endurecer los umbrales__ para así asegurarnos, en la medida de lo posible, que las palabras resultantes serán de mayor calidad:

In [284]:
umbral_repetición = 6
umbral_longitud = 5

Ahora es el momento de ejecutar el pequeño algoritmo para auto-generar las palabras clave __si faltan__, para cada categoría:

In [285]:
for categoría in categorías:
    if (categoría not in palabras_clave_categoría) or (len(palabras_clave_categoría[categoría]) == 0):
        palabras_clave_categoría[categoría] = genera_palabras_clave(categoría)

Comprobamos las palabras clave de cada categoría:

In [286]:
print("Palabras clave por categoría:")

for categoría in categorías:
    print("----------")
    print("Palabras clave de [%s]:" % (categoría))
    print(palabras_clave_categoría[categoría])

Palabras clave por categoría:
----------
Palabras clave de [j]:
['tokyo', 'japon', 'chino']
----------
Palabras clave de [ch]:
['chino', 'shanghai', 'pekin', 'macao']


Y, a continuación, añadimos todas las palabras clave de cada categoría a un nuevo conjunto que contenga __todas las palabras clave__.

In [287]:
palabras_clave = set()

# Update nos permite añadir el contenido de un set a otro set
for categoría in categorías:
    palabras_clave.update(palabras_clave_categoría[categoría])

print("Las palabras clave son:\n%s" % (palabras_clave))

Las palabras clave son:
{'japon', 'chino', 'macao', 'shanghai', 'tokyo', 'pekin'}


Y, antes de acabar, realizaremos un pequeño estudio relacionado con las palabras clave seleccionadas:

In [288]:
print("Número total de palabras clave: \t %s" % (len(palabras_clave)))
print("Media de palabras clave por categoría: \t %s" % (len(palabras_clave)/len(categorías)))

Número total de palabras clave: 	 6
Media de palabras clave por categoría: 	 3.0


## Parte IV: Procesamiento

En esta parte se va a llevar a cabo el procesamiento de datos para posteriormente utilizarlos en los algoritmos de __Naive Bayes__ y __kNN__.

Puesto que se deben realizar cálculos distintos para cada algoritmo, dividiremos esta sección en dos subsecciones:

### Parte IV-A: Procesamiento de Naive Bayes

Para aplicar el algoritmo Naive Bayes primero debemos calcular todos los __P(c)__ ___(probabilidad de "c")___ y los __P(t|c)__ ___(probabilidad de "t" condicionada a "c")___.

En este caso __"c"__ sería nuestra categoría y __"t"__ cada palabra clave.

Primero, vamos a calcular los __P(c)__. Para ello, tenemos que contar el número de documentos de la categoría en cuestión existentes en nuestro conjunto de entrenamiento y dividirlo entre el número total de documentos de nuestro conjunto de entrenamiento. Así pues, por ejemplo, la probabilidad de acción (__P(acción)__) sería el número resultante de dividir el total de documentos catalogados como "acción" de nuestro conjunto de entrenamiento entre el número total de documentos del conjunto de entrenamiento.

Como tenemos un conjunto con todos los archivos de entrenamiento y un conjunto específico por cada categoría:

In [289]:
probabilidad_categoría = {}

for categoría in categorías:
    probabilidad_categoría[categoría] = len(archivos_entrenamiento_categoría[categoría]) / len(archivos_entrenamiento)
    print("P(%s) = \t %f" % (categoría, probabilidad_categoría[categoría]))

P(j) = 	 0.250000
P(ch) = 	 0.750000


Sólo para asegurarnos, todas las probabilidades deben sumar __~1__ en este apartado:

In [290]:
print("Suma de probabilidades (debe ser ~1) \t %f" % (sum([probabilidad_categoría[categoría] for categoría in categorías])))

Suma de probabilidades (debe ser ~1) 	 1.000000


Ahora, para calcular los __P(t|c)__ será un poco más complejo. Para llevar a cabo esta tarea haremos uso de __un diccionario por cada categoría__ que __relacionará palabras clave con su probabilidad condicionada a la categoría del diccionario__. De nuevo, tomaremos la categoría "acción" como ejemplo: el diccionario "probabilidad_palabraclave_acción" recogerá, por cada palabra clave, su probabilidad condiccionada a la categoría acción, es decir, su __P(t|acción__), siendo "t" cada entrada del diccionario.

Pero antes de empezar, definiremos un método, que utilizaremos en los siguientes pasos, que genere cada diccionario deseado como salida y, además, le __aplique un suavizado de LaPlace__:

In [291]:
def crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_categoría, palabras_clave_categoría):
    probabilidades_condicionadas = {}
    cuenta_palabras_categoría = cuenta_palabras_desde_archivos(ruta_conjunto_entrenamiento, archivos_entrenamiento_categoría) # Almacena las palabras clave de la categoría y el número de veces que se repiten.
    
    número_palabras_clave_totales = len(palabras_clave) # Número de palabras clave que poseemos en total.
    número_palabras_clave_categoría = sum(cuenta_palabras_categoría.values())
    
    for palabra_clave in palabras_clave:
        if palabra_clave in cuenta_palabras_categoría:
            número_veces_aparece_palabra_en_categoría = cuenta_palabras_categoría[palabra_clave] # Número de veces que la palabra clave se repite en esta categoría.
        else:
            número_veces_aparece_palabra_en_categoría = 0
        result = ((número_veces_aparece_palabra_en_categoría + 1) / (número_palabras_clave_categoría + número_palabras_clave_totales)) # Añadimos 1 en el numerador y el número de palabras claves totales en el denominador para aplicar el suavizado.
        probabilidades_condicionadas[palabra_clave] = result
    
    return probabilidades_condicionadas

También vamos a crear un pequeño método que nos ayude a visualizar las probabilidades condicionadas:

In [292]:
def mostrar_diccionario_probabilidades_condicionadas(diccionario, categoría):
    for entrada in diccionario:
        print("P(%s|%s) = \t %f" % (entrada, categoría, diccionario[entrada]))

Ahora, empezamos con el cálculo en sí:

In [293]:
probabilidad_palabraclave = {}

for categoría in categorías:
    probabilidad_palabraclave[categoría] = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_categoría[categoría], palabras_clave_categoría[categoría])
    
    print("----- %s -----" % (categoría))
    mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave[categoría], categoría)

----- j -----
P(japon|j) = 	 0.222222
P(chino|j) = 	 0.222222
P(macao|j) = 	 0.111111
P(shanghai|j) = 	 0.111111
P(tokyo|j) = 	 0.222222
P(pekin|j) = 	 0.111111
----- ch -----
P(japon|ch) = 	 0.071429
P(chino|ch) = 	 0.428571
P(macao|ch) = 	 0.142857
P(shanghai|ch) = 	 0.142857
P(tokyo|ch) = 	 0.071429
P(pekin|ch) = 	 0.142857


Una vez calculadas todas las probabilidades condicionadas de todas las categorías ya hemos finalizado con este subapartado, pero antes vamos a recordar un par de cosas:

Para acceder a una probabilidad específica símplemente ejecutamos lo siguiente (para el ejemplo obtendremos la __probabilidad de coche__ condicionada a la categoría __acción__, es decir, __P(coche|acción)__):

In [294]:
print("P(coche|acción) = %f" % (probabilidad_palabraclave["acción"]["coche"]))

Si hubiese una __palabra clave que no apareciese en acción, la probabilidad no sería 0__ ya que estamos usando suavizado. Por ejemplo, podemos comprobarlo con la probabilidad de soldado (que no forma parte de las palabras clave de acción) condicionada a acción (__P(soldado|acción)__):

In [295]:
print("P(gags|acción) = %f" % (probabilidad_palabraclave["acción"]["gags"]))

### Parte IV-B: Procesamiento de kNN

In [296]:
palabras_clave_y_repeticiones = {}

cuenta_palabras = cuenta_palabras_desde_archivos(ruta_conjunto_entrenamiento, archivos_entrenamiento)

for palabra in cuenta_palabras:
    if palabra in palabras_clave:
        palabras_clave_y_repeticiones[palabra] = cuenta_palabras[palabra]

print(palabras_clave_y_repeticiones)

{'chino': 6, 'shanghai': 1, 'macao': 1, 'tokyo': 1, 'japon': 1, 'pekin': 1}


In [297]:
def calculo_peso(palabra_clave, ruta, archivo):
    frecuencia_en_documento = 0
    frecuencia_documental = 0
    frecuencia_documental_inversa = 0
    peso = 0
    
    cuenta_palabras = cuenta_palabras_desde_archivo(ruta, archivo)
    if palabra_clave in cuenta_palabras:
        frecuencia_en_documento = cuenta_palabras[palabra_clave]
    
    frecuencia_documental = palabras_clave_y_repeticiones[palabra_clave]
    
    if frecuencia_documental is not 0:
        aux = len(archivos_entrenamiento) / frecuencia_documental
    else:
        aux = 0
    
    frecuencia_documental_inversa = math.log(aux)
    
    peso = frecuencia_en_documento * frecuencia_documental_inversa
    
    return peso

In [298]:
def calculo_pesos(ruta, archivo):
    lista_pesos = []
    
    for palabra_clave in palabras_clave:
        lista_pesos.append(calculo_peso(palabra_clave, ruta, archivo))
    
    return lista_pesos

In [299]:
import math

diccionario_palabraclave_pesos = {}

for archivo in archivos_entrenamiento:
    lista_pesos_archivo = calculo_pesos(ruta_conjunto_entrenamiento, archivo)
    
    diccionario_palabraclave_pesos[archivo] = lista_pesos_archivo

# print(diccionario_palabraclave_pesos)

## Parte V: Salvado del procesamiento en fichero

### Parte V-A: Salvado del procesamiento de Naive Bayes

Dado que el enunciado de la práctica requiere que guardemos el procesado que acabamos que realizar (en la parte IV) para después utilizarlo en la ejecución de los algoritmos, procedemos a ello.

Primero necesitamos crear dos grandes listas: la primera contendrá la cadena relacionada con la probabilidad de que pase algo y la segunda la probabilidad de que pase. Ambas listas deben ser iguales y debemos almacenar toda la información obtenida.

In [300]:
lista_texto = []
lista_valor = []

for categoría in categorías:
    lista_texto.append("P(" + categoría + ")")
    lista_valor.append(probabilidad_categoría[categoría])
    
    for entrada in probabilidad_palabraclave[categoría]:
        lista_texto.append("P(" + entrada + "|" + categoría + ")")
        lista_valor.append(probabilidad_palabraclave[categoría][entrada])

In [301]:
import csv # Librería que necesitaremos para guardar en formato ".csv".

csvfile = "csv/naive-bayes.csv"
datos_a_guardar = zip(lista_texto, lista_valor)

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for dato in datos_a_guardar:
        writer.writerow([dato[0]] + [dato[1]])

### Parte V-B: Salvado del procesamiento de kNN

In [302]:
csvfile = "csv/knn.csv"
datos_a_guardar = diccionario_palabraclave_pesos

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for dato in datos_a_guardar:
        writer.writerow([dato] + [datos_a_guardar[dato]])

## Parte VI: Ejecución de los algoritmos

En la parte II ya encontramos los ficheros del conjunto de test, que están en su respectiva carpeta, ahora tenemos que procesaros igual que hicimos en la parte III con los ficheros del conjunto de pruebas.

Vamos a definir un método que nos será de utilidad: a partir de un __csv__ dado y de una __cadena a buscar__, devolverá el valor relacionado con dicha celda del csv. Esto nos será útil para, a partir de la cadena de una probabilidad (por ejemplo, _P(bala|acción)_), nos devuelva su probabilidad.

In [303]:
def lee_fichero(nombre_csv):
    with open(nombre_csv, 'rt', encoding="latin-1") as fichero:
        lector = csv.reader(fichero)
        diccionario = dict(lector)
    
    return diccionario

### Parte VI-A: Ejecución de Naive Bayes

El método naive_bayes recibe un archivo y los datos procesados (probabilidades) como parámetros y determina la categoría del archivo pasado como parámetro.

In [314]:
def naive_bayes(archivo, csv):
    cuenta_palabras = cuenta_palabras_desde_archivo(ruta_conjunto_prueba, archivo)
    
    palabras_coincidentes_con_palabras_clave = cuenta_palabras.copy()
    
    # De las palabras que contiene el fichero, desechamos todas las que no coinciden con las palabras clave.
    for palabra in cuenta_palabras:
        if palabra not in palabras_clave:
            del palabras_coincidentes_con_palabras_clave[palabra]
    
    # Abrimos el fichero ".csv" generado para consultar datos en el siguiente paso.
    datos = lee_fichero(csv)
    
    # Ejecutamos el algoritmo en sí.
    resultados = {} # Resultados será un diccionario que contendrá la categoría y la "puntuación" otorgada por el algoritmo a esa categoría (para posteriormente elegir la categoría con el máximo valor).
    
    for categoría in categorías:
        probabilidades_condicionadas_a_multiplicar = []
        for palabra_clave in palabras_coincidentes_con_palabras_clave:
            cadena_a_buscar = "P(" + palabra_clave + "|" + categoría + ")" # Define la cadena que se debe buscar en el archivo. En este caso la probabilidad condicionada a la categoría.
            probabilidades_condicionadas_a_multiplicar.append(float(datos[cadena_a_buscar]) ** palabras_coincidentes_con_palabras_clave[palabra_clave]) # Busca el valor de la probabilidad condicionada requerida (lo transforma en float), lo eleva al número de veces que se repite y añade el valor de la probabilidad condicionada hallada a una lista que se pasará a multiplicar después.
        
        cadena_a_buscar = "P(" + categoría + ")" # Define la cadena que se debe buscar en el archivo. En este caso la probabilidad de la categoría.
        probabilidad_categoría = float(datos[cadena_a_buscar])
        
        # Multiplicamos los elementos de la lista de probabilidades condicionadas
        probabilidades_condicionadas_multiplicadas = 1.0
        for elemento in probabilidades_condicionadas_a_multiplicar:
            probabilidades_condicionadas_multiplicadas = probabilidades_condicionadas_multiplicadas * elemento
        
        resultados[categoría] = probabilidades_condicionadas_multiplicadas * probabilidad_categoría # Calcula el coeficiente
    
    result = max(resultados, key=resultados.get) # Devuleve el resultado del algoritmo. En este caso el elemento del diccionario con mayor coeficiente.
    
    return result

__Ejecutamos el algoritmo__ e __imprimimos__ los __resultados__ obtenidos:

In [315]:
def aplicar_naive_bayes_archivos_prueba():
    print("El resultado de aplicar el algoritmo [Naive-Bayes] al conjunto de pruebas es...")
    for archivo in archivos_prueba:
        algoritmo = naive_bayes(archivo, "csv/naive-bayes.csv")
        print("[%s] \t [%s]" % (archivo.replace(".txt", ""), algoritmo))

In [316]:
aplicar_naive_bayes_archivos_prueba()

El resultado de aplicar el algoritmo [Naive-Bayes] al conjunto de pruebas es...
[A todo gas - Tokyo race] 	 [ch]


### Parte VI-B: Ejecución de kNN

In [307]:
def calcula_distancia(v, w):
    # Comprueba que las listas son del mismo tamaño.
    if(len(v) == len(w)):
        numerador = sum([elemento_v * elemento_w for elemento_v, elemento_w in zip(v,w)])
        
        denominador_parte_v = math.sqrt(sum([elemento_v ** 2 for elemento_v in v]))
        denominador_parte_w = math.sqrt(sum([elemento_w ** 2 for elemento_w in w]))
        
        denominador = denominador_parte_v * denominador_parte_w
        
        return numerador / denominador

In [308]:
def knn(archivo, csv):
    v = calculo_pesos(ruta_conjunto_prueba, archivo)
    
    # Abrimos el fichero ".csv" generado para consultar datos en el siguiente paso.
    datos = lee_fichero(csv)
    
    # Ejecutamos el algoritmo en sí.
    resultados = {} # Resultados será un diccionario que contendrá el archivo y la "puntuación" (similitud) otorgada por el algoritmo a esa categoría (para posteriormente elegir la categoría del archivo con la similitud más cercana a uno, que será el mayor valor).
    
    # Ahora que tenemos el peso del archivo a clasificar mediante el algoritmo y los pesos de los archivos del conjunto de entrenamiento (extraídos del ".csv" y guardados en forma de diccionario) tenemos que calcular, una por una, la distancia a cada elemento del conjunto de entrenamiento y quedarnos con la menor.
    for dato in datos:
        # Como lo que guardamos es una cadena, es necesario un pequeño procesamiento para transformarlo de nuevo en una lista.
        w = datos[dato].replace('[', '') # Primero eliminamos "[".
        w = w.replace(']', '') # Hacemos lo mismo con "]".
        w = w.split(",") # Aplicamos ".split()" para volver a "trocear" la cadena y convertirla de nuevo en una lista.
        w = [float(elemento) for elemento in w]
        resultados[dato] = calcula_distancia(v, w) # "v" son los pesos del archivo a clasificar y "w" los del archivo del conjunto de entrenamiento que está siendo procesado.
    
    # Para una mayor exactitud, saber en qué categoría se enmarcará la muestra y a qué película se debe, usaremos una lista para devolver dicha información (el elemento 0 contendrá la categoría y el 1 la película de dónde procede).
    result = max(resultados, key=resultados.get)
    result = result.split("/")
    
    return result[0], result[1]

In [309]:
def aplicar_knn_archivos_prueba():
    print("El resultado de aplicar el algoritmo [kNN] al conjunto de pruebas es...")
    for archivo in archivos_prueba:
        algoritmo = knn(archivo, "csv/knn.csv")
        print("[%s] \t [%s] (por similitud con [%s])" % (archivo.replace(".txt", ""), algoritmo[0], algoritmo[1].replace(".txt", "")))

In [310]:
aplicar_knn_archivos_prueba()

El resultado de aplicar el algoritmo [kNN] al conjunto de pruebas es...
[A todo gas - Tokyo race] 	 [j] (por similitud con [Apocalipsis now])


## Parte VII: Análisis de los resultados

## Parte VIII: Conclusiones